# Train gradient boosting model

# 1. Imports

## 1.1 Packages

In [ ]:
import sys

import pandas as pd


## 1.2 Options

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
sys.path.append('../src')

from velib_prediction.pipelines.train_model.mlflow import (  # noqa: E402
    create_mlflow_experiment,
)
from velib_prediction.pipelines.train_model.nodes import (  # noqa: E402
    get_split_train_val_cv,
    train_model_cv_mlflow,
)


## 1.3 Datasets

In [ ]:
df_train = pd.read_parquet("../data/04_feature/df_feat_train.parquet")
df_train.sample(2)

# 2. Prepare datasets

In [ ]:
df_train.drop(
    columns=[
        "duedate", "coordonnees_geo", "date"
    ],
    inplace=True
)

df_train.rename(columns={"numbikesavailable": "target"}, inplace=True)

In [ ]:
list_df = get_split_train_val_cv(df_train, n_splits=3)

In [ ]:
len(list_df)

In [ ]:
feat_cat = [
    "is_installed",
    "is_renting",
    "is_returning",
    "code_insee_commune",
    "duedate_weekend",
]

# 3. Train model

In [ ]:
experiment_id = create_mlflow_experiment(
    experiment_folder_path="../data/06_models/mlruns",
    experiment_name="velib_prediction"
)
experiment_id

In [ ]:
params_catboost = {
    "iterations": 100,
    "depth": 7,
}

In [ ]:
train_model_cv_mlflow(
    run_name="Test_catboost",
    experiment_id=experiment_id,
    list_train_valid=list_df,
    feat_cat=feat_cat,
    verbose=10,
    **params_catboost
)